In [1]:
import torch
import warnings
import numpy as np
from opacus import PrivacyEngine
from torch import Tensor, nn, optim
import torch.nn.functional as F
from pistacchio_simulator.Models.mnist import MnistNet
import argparse

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from opacus import PrivacyEngine
from torchvision import datasets, transforms
from tqdm import tqdm

In [2]:
MAX_GRAD_NORM = 100000.0
NOISE = 1.0
DELTA = 1e-5
EPOCHS = 10

LR = 0.01
BATCH_SIZE = 128
MAX_PHYSICAL_BATCH_SIZE = 128

In [3]:
class SampleConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 4, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # x of shape [B, 1, 28, 28]
        x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.relu(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x

    def name(self):
        return "SampleConvNet"

In [4]:
data = torch.load(
    "../examples/data/mnist/federated_data/cluster_0_node_0_private_train.pt"
)

train_loader = torch.utils.data.DataLoader(
    data,
    batch_size=128,
    shuffle=False,
    num_workers=0,
)

data_test = torch.load("../examples/data/mnist/federated_data/server_validation_set.pt")

test_loader = torch.utils.data.DataLoader(
    data_test,
    batch_size=128,
    shuffle=False,
    num_workers=0,
)

# MNIST_MEAN = 0.1307
# MNIST_STD = 0.3081

# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST(
#         "../mnist",
#         train=True,
#         download=True,
#         transform=transforms.Compose(
#             [
#                 transforms.ToTensor(),
#                 transforms.Normalize((MNIST_MEAN,), (MNIST_STD,)),
#             ]
#         ),
#     ),
#     batch_size=128,
#     num_workers=0,
#     pin_memory=True,
# )

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda', index=0)

In [7]:
privacy_engine = PrivacyEngine(accountant="rdp")

/home/lcorbucci/.cache/pypoetry/virtualenvs/pistacchio-fl-simulator-XKOu9Fs5-py3.10/lib/python3.10/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [8]:
# # model = SampleConvNet()
# # model = MnistNet()

# model = model.to(device)
# # optimizer = optim.RMSprop(
# #     model.parameters(),
# #     lr=0.1,
# # )

In [9]:
def accuracy(preds, labels):
    return (preds == labels).mean()


def train(model, device, train_loader, optimizer, privacy_engine, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()

        acc = accuracy(preds, labels)
        losses.append(loss.item())
        top1_acc.append(acc)

        epsilon = privacy_engine.accountant.get_epsilon(delta=DELTA)
        if (_batch_idx + 1) % 10 == 0:
            print(
                f"Train Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"(ε = {epsilon:.2f}, δ = {DELTA})"
                f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
            )

In [10]:
def test(model, device, test_loader):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    return correct / len(test_loader.dataset)

In [11]:
# # (
# #     model,
# #     optimizer,
# #     train_loader,
# # ) = privacy_engine.make_private(
# #     module=model,
# #     optimizer=optimizer,
# #     data_loader=train_loader,
# #     noise_multiplier=1.0,
# #     max_grad_norm=1000000,
# # )
# (
#     model,
#     optimizer,
#     train_loader,
# ) = privacy_engine.make_private_with_epsilon(
#     module=model,
#     optimizer=optimizer,
#     data_loader=train_loader,
#     epochs=10,
#     target_epsilon=5,
#     target_delta=DELTA,
#     max_grad_norm=1000000,
# )

In [12]:
# for i in range(10):
#     train(model, train_loader, optimizer, i, device, privacy_engine, DELTA)

In [13]:
model = MnistNet().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0)
privacy_engine = None

privacy_engine = PrivacyEngine(accountant="rdp")
model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=1.0,
    max_grad_norm=10.0,
)

for epoch in range(0, 10):
    train(model, device, train_loader, optimizer, privacy_engine, epoch)

  0%|          | 0/235 [00:00<?, ?it/s]/home/lcorbucci/.cache/pypoetry/virtualenvs/pistacchio-fl-simulator-XKOu9Fs5-py3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
  6%|▌         | 13/235 [00:02<00:19, 11.20it/s]

Train Epoch: 0 	Loss: 2.241744 (ε = 0.83, δ = 1e-05)Acc@1: 20.086136 


  9%|▉         | 22/235 [00:02<00:14, 14.53it/s]

Train Epoch: 0 	Loss: 2.095282 (ε = 0.84, δ = 1e-05)Acc@1: 30.205029 


 14%|█▍        | 33/235 [00:03<00:12, 16.15it/s]

...

Train Epoch: 9 	Loss: 1.452645 (ε = 1.35, δ = 1e-05)Acc@1: 74.118756 


 95%|█████████▌| 224/235 [00:08<00:00, 25.10it/s]

Train Epoch: 9 	Loss: 1.451330 (ε = 1.35, δ = 1e-05)Acc@1: 74.152801 


100%|██████████| 235/235 [00:09<00:00, 25.58it/s]

Train Epoch: 9 	Loss: 1.454008 (ε = 1.35, δ = 1e-05)Acc@1: 74.222126 


In [14]:
test(model, device, test_loader)

100%|██████████| 79/79 [00:02<00:00, 38.00it/s]


Test set: Average loss: 0.0063, Accuracy: 8367/10000 (83.67%)



0.8367

In [15]:
# model = SampleConvNet().to(device)

# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0)
# privacy_engine = None

# privacy_engine = PrivacyEngine(accountant="rdp")
# model, optimizer, train_loader = privacy_engine.make_private(
#     module=model,
#     optimizer=optimizer,
#     data_loader=train_loader,
#     noise_multiplier=1.0,
#     max_grad_norm=100000.0,
# )

# for epoch in range(0, 10):
#     train(model, device, train_loader, optimizer, privacy_engine, epoch)

In [16]:
# test(model, device, test_loader)